# **Import Functions**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# **Reading the Data**

In [ ]:
# Make use of the main google sheet for accessing the data
def read_google_sheet(sheetId, sheetName):
  SHEET_ID = sheetId
  SHEET_NAME = sheetName
  url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'

  return url

# replace the google_sheet_token with the token of your desired google sheet from the link. Replace the name_of_sheet with the name of the sheet you want to access.
addresses = pd.read_csv(read_google_sheet('google_sheet_token', 'name_of_sheet'), index_col=0)
weather = pd.read_csv(read_google_sheet('google_sheet_token', 'name_of_sheet'), index_col=0)
meal_counts = pd.read_csv(read_google_sheet('google_sheet_token', 'name_of_sheet'), index_col=0)
unserved_people = pd.read_csv(read_google_sheet('google_sheet_token', 'name_of_sheet'), index_col=0)
congvsnoncong = pd.read_csv(read_google_sheet('google_sheet_token', 'name_of_sheet'), index_col=0)
event = pd.read_csv(read_google_sheet('google_sheet_token', 'name_of_sheet'), index_col=0)
data_2019 = pd.read_csv(read_google_sheet('google_sheet_token', 'name_of_sheet'), index_col=0)
region_size = pd.read_csv(read_google_sheet('google_sheet_token', 'name_of_sheet'), index_col=0)
data_2023 = pd.read_csv(read_google_sheet('google_sheet_token', 'name_of_sheet'), index_col=0)

# make use of the google form sheet link for adding new data moving forward
new_url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
new_response = pd.read_csv(new_url, index_col = 0)


In [ ]:
addresses = addresses.reset_index()
weather = weather.reset_index()
meal_counts = meal_counts.reset_index()
unserved_people = unserved_people.reset_index()
congvsnoncong = congvsnoncong.reset_index()
event = event.reset_index()
data_2019 = data_2019.reset_index()
region_size = region_size.reset_index()
data_2023 = data_2023.reset_index()
new_response = new_response.reset_index()

# **Overview of the Data**

In [ ]:
data = [addresses, weather, meal_counts, unserved_people, congvsnoncong, data_2019, new_response]

for df in data:
  print("__________________________________________________________________________________________________________________")
  print(f"Shape: {df.shape}")
  print(f"Columns: {df.columns}")
  print(f"Info: {df.info()}")
  print("Sample records:")
  print(df.head(3).to_string(index=False))
  print("__________________________________________________________________________________________________________________")



# **Data Analysis**

In [ ]:
def years_shape(x, col):
  years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

  shapes = []
  for year in years:
    shapes.append(x[x[col] == year].shape)

  return shapes

print(meal_counts.shape)
print(years_shape(meal_counts, 'Year'))

meal_counts = meal_counts.drop('Timestamp', axis=1).drop_duplicates()

print(meal_counts.shape)
print(years_shape(meal_counts, 'Year'))

years = [2016, 2017, 2018, 2019, 2020, 2021, 2022]
meal_counts1 = meal_counts[meal_counts['Year'].isin(years)]

print(meal_counts1.shape)
print(years_shape(meal_counts1, 'Year'))

In [ ]:
def validation1(x):
  before_operation = x['Number of Meals Received / Prepared'] + x['Meals available from previous day']
  after_operation = x['total number of meals served'] + x['Total damaged/incomplete/other non-reimbursable meals'] + x['Total leftover meals']

  return before_operation != after_operation

def validation2(x):
  before_operation = x['Number of Meals Received / Prepared'] + x['Meals available from previous day']
  after_operation = x['Total number of first meals '] + x['Total number second meals'] + x['total program adult meals'] + x['total non-program adult meals'] + x['Total damaged/incomplete/other non-reimbursable meals'] + x['Total leftover meals']

  return before_operation != after_operation

def validation3(x):
  before_operation = x['total number of meals served']
  after_operation = x['Total number of first meals '] + x['Total number second meals'] + x['total program adult meals'] + x['total non-program adult meals']

  return before_operation != after_operation


In [ ]:
error1 = meal_counts1.apply(validation1, axis=1)
error2 = meal_counts1.apply(validation2, axis=1)
error3 = meal_counts1.apply(validation3, axis=1)
meal_counts1['error'] = error1 | error2 | error3

meal_counts2 = meal_counts1[meal_counts1['error']==False]

print(meal_counts2.shape)
print(years_shape(meal_counts2, 'Year'))


In [ ]:
meal_counts3 = meal_counts2[(meal_counts2['Site Name'].notnull()) & (meal_counts2['Date'].notnull())]

print(meal_counts3.shape)
print(years_shape(meal_counts3, 'Year'))

In [ ]:
meal_counts4 = meal_counts3.drop_duplicates(['Site Name', 'Date', 'Meal Type', 'Delivery Time',
        'Number of Meals Received / Prepared',
        'Meals available from previous day', 'Total number of first meals ',
        'Total number second meals', 'total program adult meals',
        'total non-program adult meals', 'total number of meals served',
        'Total damaged/incomplete/other non-reimbursable meals',
        'Total leftover meals'])

print(meal_counts4.shape)
print(years_shape(meal_counts4, 'Year'))

In [ ]:
meal_counts4['date'] = pd.to_datetime(meal_counts4['Date'], errors='coerce')
meal_counts4['week_number'] = meal_counts4['date'].dt.isocalendar().week

meal_counts4.columns

In [ ]:
meal_unserved_counts = pd.merge(meal_counts4, unserved_people[['Name Of Site', 'Number of additional children requesting meals after all available meals were served:', 'Date', 'Meal Type']], how='left', left_on=['Site Name', 'Date', 'Meal Type'], right_on=['Name Of Site', 'Date', 'Meal Type'])

print(meal_unserved_counts.shape)
print(years_shape(meal_unserved_counts, 'Year'))

In [ ]:
import math

def divide_meals_by_5(counts, meal_numbers):

  for column in meal_numbers:
    counts[column] = counts[column].apply(lambda x: math.ceil(x / 5))


In [ ]:
def data_cleaning_pipeline_cong(df):
  print("Initial Data: ",df.shape)
  error1 = df.apply(validation1, axis=1)
  error2 = df.apply(validation2, axis=1)
  error3 = df.apply(validation3, axis=1)
  df['error'] = error1 | error2 | error3

  df = df[df['error']==False]

  print("Data after error validation: ",df.shape)

  df1 = df[(df['Site Name'].notnull()) & (df['Date'].notnull())]

  print("Data after empty site names and dates removed: ",df1.shape)

  df2 = df1[['Site Name', 'Date', 'Meal Type',
       'Number of Meals Received / Prepared',
       'Meals available from previous day', 'Total number of first meals ',
       'Total number second meals', 'total program adult meals',
       'total non-program adult meals', 'total number of meals served',
       'Total damaged/incomplete/other non-reimbursable meals',
       'Total leftover meals',
       'Number of additional children requesting meals after all available meals were served:',
       'error']]

  df3 = df2.drop_duplicates(['Site Name', 'Date', 'Meal Type',
       'Number of Meals Received / Prepared',
       'Meals available from previous day', 'Total number of first meals ',
       'Total number second meals', 'total program adult meals',
       'total non-program adult meals', 'total number of meals served',
       'Total damaged/incomplete/other non-reimbursable meals',
       'Total leftover meals',
       'Number of additional children requesting meals after all available meals were served:'])

  print("Data after dropping duplicates: ",df3.shape)

  df3['date'] = pd.to_datetime(df3['Date'], errors='coerce')
  df3['week_number'] = df3['date'].dt.isocalendar().week

  df3['Year'] = df3['date'].dt.year
  df3['Day of Week'] = (df3['date'].dt.dayofweek + 1) % 7 + 1

  return df3

In [ ]:
cleaned_2019_data = data_cleaning_pipeline_cong(data_2019)
cleaned_2019_data.columns

In [ ]:
def data_cleaning_pipeline_noncong(df):
  print("Initial Data: ",df.shape)
  error1 = df.apply(validation1, axis=1)
  error2 = df.apply(validation2, axis=1)
  error3 = df.apply(validation3, axis=1)
  df['error'] = error1 | error2 | error3

  df = df[df['error']==False]

  print("Data after error validation: ",df.shape)

  df1 = df[(df['Site Name'].notnull()) & (df['Date'].notnull())]

  print("Data after empty site names and dates removed: ",df1.shape)

  df2 = df1[['Site Name', 'Date', 'Meal Type',
       'Number of Meals Received / Prepared',
       'Meals available from previous day', 'Total number of first meals ',
       'Total number second meals', 'total program adult meals',
       'total non-program adult meals', 'total number of meals served',
       'Total damaged/incomplete/other non-reimbursable meals',
       'Total leftover meals',
       'Number of additional children requesting meals after all available meals were served:',
       'error']]

  df3 = df2.drop_duplicates(['Site Name', 'Date', 'Meal Type',
       'Number of Meals Received / Prepared',
       'Meals available from previous day', 'Total number of first meals ',
       'Total number second meals', 'total program adult meals',
       'total non-program adult meals', 'total number of meals served',
       'Total damaged/incomplete/other non-reimbursable meals',
       'Total leftover meals',
       'Number of additional children requesting meals after all available meals were served:'])

  print("Data after dropping duplicates: ",df3.shape)

  meal_numbers = ['Number of Meals Received / Prepared',
       'Meals available from previous day', 'Total number of first meals ',
       'Total number second meals', 'total program adult meals',
       'total non-program adult meals', 'total number of meals served',
       'Total damaged/incomplete/other non-reimbursable meals',
       'Total leftover meals',
       'Number of additional children requesting meals after all available meals were served:']

  divide_meals_by_5(df3, meal_numbers)

  df3['date'] = pd.to_datetime(df3['Date'], errors='coerce')
  df3['week_number'] = df3['date'].dt.isocalendar().week

  df3['Year'] = df3['date'].dt.year
  df3['Day of Week'] = (df3['date'].dt.dayofweek + 1) % 7 + 1

  return df3

In [ ]:
cleaned_2023_data = data_cleaning_pipeline_noncong(data_2023)
cleaned_2023_data

In [ ]:
cleaned_new_response = data_cleaning_pipeline_noncong(new_response)
cleaned_new_response

In [ ]:
meal_unserved_counts = meal_unserved_counts[['Site Name',
       'Date', 'Meal Type',
       'Number of Meals Received / Prepared',
       'Meals available from previous day', 'Total number of first meals ',
       'Total number second meals', 'total program adult meals',
       'total non-program adult meals', 'total number of meals served',
       'Total damaged/incomplete/other non-reimbursable meals',
       'Total leftover meals', 'Year',
       'Day of Week', 'error', 'date', 'week_number',
       'Number of additional children requesting meals after all available meals were served:']]

In [ ]:
meal_unserved_counts1 = pd.concat([meal_unserved_counts, cleaned_2019_data], ignore_index=True)

print(meal_unserved_counts1.shape)
print(years_shape(meal_unserved_counts1, 'Year'))

In [ ]:
meal_unserved_counts2 = pd.concat([meal_unserved_counts1, cleaned_new_response], ignore_index=True)

print(meal_unserved_counts2.shape)
print(years_shape(meal_unserved_counts2, 'Year'))

In [ ]:
meal_counts_data = pd.concat([meal_unserved_counts2, cleaned_2023_data], ignore_index=True)

print(meal_counts_data.shape)
print(years_shape(meal_counts_data, 'Year'))

In [ ]:
adddresses_notnull = addresses[addresses['Region'].notna()]
addresses[addresses['Region'].isna()]

In [ ]:
regions = pd.merge(adddresses_notnull[['Name Of Site', 'Address', 'Zip Code', 'County', 'Region']], region_size[['region', 'size']],left_on=['Region'], right_on=['region'])

In [ ]:
meals_dataset = pd.merge(meal_counts_data, regions[['Name Of Site', 'Address', 'Zip Code', 'County', 'Region', 'size']], left_on=['Site Name'], right_on=['Name Of Site'])

print(meals_dataset.shape)
print(years_shape(meals_dataset, 'Year'))

In [ ]:
meal_counts_data[meal_counts_data['Site Name'] == 'Town Creek'].shape

In [ ]:
people_counts1 = meals_dataset[['Site Name', 'Date', 'Meal Type',
       'Number of Meals Received / Prepared', 'week_number',
       'Meals available from previous day', 'Total number of first meals ',
       'Total number second meals', 'total program adult meals',
       'total non-program adult meals', 'total number of meals served',
       'Total damaged/incomplete/other non-reimbursable meals',
       'Total leftover meals', 'Year', 'Number of additional children requesting meals after all available meals were served:',
       'Day of Week', 'Address', 'Zip Code', 'County', 'Region', 'size']]

people_counts1.rename(columns={'Site Name':'site_name', 'Date':'date', 'Meal Type' : 'meal_type',
                            'Number of Meals Received / Prepared': 'ordered_meals', 'Meals available from previous day': 'previous_day_meals',
                            'Total number of first meals ': 'first_meals', 'Total number second meals': 'second_meals',
                            'total program adult meals': 'program_adult_meals', 'total non-program adult meals': 'nonprogram_adult_meals',
                            'total number of meals served': 'served_meals', 'Total damaged/incomplete/other non-reimbursable meals': 'damaged_meals',
                            'Total leftover meals': 'leftover_meals', 'Year': 'year', 'Day of Week': 'day_of_week',
                            'Number of additional children requesting meals after all available meals were served:': 'unserved_people',
                            'Address':'address', 'Zip Code': 'zip_code', 'County': 'county',
                            'Region': 'region'}, inplace=True)

In [ ]:
people_counts1.shape

In [ ]:
people_counts1.isna().sum()

In [ ]:
people_counts1['unserved_people'].fillna(0, inplace=True)

In [ ]:
def drop_noncong_duplicates(df):
  numerical_columns = ['ordered_meals', 'previous_day_meals',
          'first_meals', 'second_meals', 'program_adult_meals',
          'nonprogram_adult_meals', 'served_meals', 'damaged_meals',
          'leftover_meals', 'unserved_people']

  aggregations = {col: 'mean' for col in numerical_columns}
  aggregations['meal_type'] = lambda x: 'Breakfast+Lunch'

  return df.groupby(['site_name', 'date', 'year', 'day_of_week', 'week_number', 'address', 'zip_code', 'county', 'region', 'size']).agg(aggregations).reset_index()


In [ ]:
people_counts1_2020 = people_counts1[people_counts1['year'] == 2020]
people_counts1_2020.shape

In [ ]:
people_2020 = drop_noncong_duplicates(people_counts1_2020)
people_2020.shape

In [ ]:
noncong_2021 = ['Beersheba Springs Assembly', 'Coalmont Elementary School', 'Community Action Committee', 'Epiphany Mission Episcopal Church',
'Grundy Housing Authority', 'Monteagle Greene Apartments', 'Morton Memorial United Methodist Church',
'North Elementary School', 'Palmer Elementary School', 'Pelham Elementary School ', 'Sewanee Community Center',
'Swiss Memorial Elementary School ', 'Tracy City Elementary School', 'Christ Church Episcopal']

people_counts1_2021 = people_counts1[(people_counts1['year'] == 2021) & (people_counts1['site_name'].isin(noncong_2021))]
people_counts1_2021.shape

In [ ]:
people_2021 = drop_noncong_duplicates(people_counts1_2021)
people_2021.shape

In [ ]:
meal_numbers = ['ordered_meals', 'previous_day_meals',
        'first_meals', 'second_meals', 'program_adult_meals',
        'nonprogram_adult_meals', 'served_meals', 'damaged_meals',
        'leftover_meals', 'unserved_people']

divide_meals_by_5(people_2020, meal_numbers)
divide_meals_by_5(people_2021, meal_numbers)

In [ ]:
people_counts2 = people_counts1.drop(people_counts1[(people_counts1['year'] == 2020)].index)
print(people_counts2.shape)
print(years_shape(people_counts2, 'year'))

In [ ]:
people_counts3 = people_counts2.drop(people_counts2[(people_counts2['year'] == 2021) & (people_counts2['site_name'].isin(noncong_2021))].index)
print(people_counts3.shape)
print(years_shape(people_counts3, 'year'))

In [ ]:
people_counts4 = pd.concat([people_counts3, people_2020], ignore_index=True)
people_dataset = pd.concat([people_counts4, people_2021], ignore_index=True)
print(people_dataset.shape)
print(years_shape(people_dataset, 'year'))

In [ ]:
# PLEASE BE AWARE THAT EVERYTHING MENTIONED HERE IS IN PEOPLE FROM NOW ON NOT MEALS, IGNORE THE VARIABLE NAMES BEING IN MEALS

people_dataset['available_meals'] = people_dataset['ordered_meals'] + people_dataset['previous_day_meals']
people_dataset['wasted_meals'] = people_dataset['damaged_meals'] + people_dataset['leftover_meals']

In [ ]:
people_dataset.isna().sum()

In [ ]:
people_dataset1 = people_dataset.sort_values(['year', 'week_number'])
week_id = people_dataset1[['year', 'week_number']].copy()
week_id['week_serial_number'] = week_id.groupby(['year', 'week_number']).ngroup()
week_id[['week_serial_number','week_number','year']]
week_id.drop_duplicates(inplace=True)

In [ ]:
people_dataset1.isna().sum()

In [ ]:
meals_dataset1 = pd.merge(people_dataset1, week_id[['year', 'week_number', 'week_serial_number']], on=['year', 'week_number'])

In [ ]:
meals_dataset1.shape

In [ ]:
meals_dataset1.isna().sum()

In [ ]:
event['date'] = pd.to_datetime(event['date'], errors='coerce')
event['week_number'] = event['date'].dt.isocalendar().week
event['year'] = event['date'].dt.year

In [ ]:
event.head()

In [ ]:
meals_dataset2 = pd.merge(meals_dataset1, event[['year', 'week_number', 'event']], how='left', on=['year', 'week_number'])

In [ ]:
meals_dataset2.shape

In [ ]:
meals_dataset2.isna().sum()

In [ ]:
meals_dataset2['event'].fillna(0, inplace=True)
meals_dataset2.isna().sum()

In [ ]:
weather['date'] = pd.to_datetime(weather['datetime (UTC)'], errors='coerce')
weather['year'] = weather['date'].dt.year
weather['week_number'] = weather['date'].dt.isocalendar().week

In [ ]:
weather.columns

In [ ]:
weather_info = weather.groupby(['year', 'week_number'])[['temperature (degF)', 'total_precipitation (mm of water equivalent)', 'wind_speed (m/s)','humidex_index (degF)']].mean().reset_index()
weather_info

In [ ]:
sns.boxplot(weather_info['humidex_index (degF)'])

In [ ]:
threshold_humidity = np.quantile(weather_info['humidex_index (degF)'], [0.25, 0.5, 0.75])
threshold_temp= np.quantile(weather_info['temperature (degF)'], [0.25, 0.5, 0.75])
threshold_prec = np.quantile(weather_info['total_precipitation (mm of water equivalent)'], [0.25, 0.5, 0.75])
threshold_wind = np.quantile(weather_info['wind_speed (m/s)'], [0.25, 0.5, 0.75])

weather_info['humidex_index (degF)1'] = weather_info['humidex_index (degF)'].apply(lambda x: 0 if x <  threshold_humidity[0]
                                                                                  else (1 if (x <  threshold_humidity[1])
                                                                                  else (2 if x <  threshold_humidity[2] else 3)))

weather_info['temperature (degF)1'] = weather_info['temperature (degF)'].apply(lambda x: 0 if x <  threshold_temp[0]
                                                                                  else (1 if (x <  threshold_temp[1])
                                                                                  else (2 if x <  threshold_temp[2] else 3)))

weather_info['total_precipitation (mm of water equivalent)1'] = weather_info['total_precipitation (mm of water equivalent)'].apply(lambda x: 0 if x <  threshold_prec[0]
                                                                                  else (1 if (x <  threshold_prec[1])
                                                                                  else (2 if x <  threshold_prec[2] else 3)))

weather_info['wind_speed (m/s)1'] = weather_info['wind_speed (m/s)'].apply(lambda x: 0 if x <  threshold_wind[0]
                                                                                  else (1 if (x <  threshold_wind[1])
                                                                                  else (2 if x <  threshold_wind[2] else 3)))


In [ ]:
weather_info.rename(columns={'temperature (degF)1': 'temperature', 'humidex_index (degF)1': 'humidity',
                             'total_precipitation (mm of water equivalent)1': 'precipitation', 'wind_speed (m/s)1':'wind'}, inplace=True)

In [ ]:
weather_info.drop(['temperature (degF)',
       'total_precipitation (mm of water equivalent)', 'wind_speed (m/s)',
       'humidex_index (degF)'], axis=1, inplace=True)
weather_info.columns

In [ ]:
meals_dataset2.isna().sum()

In [ ]:
weather_info.shape

In [ ]:
dataset = pd.merge(meals_dataset2, weather_info, on=['year', 'week_number'])

In [ ]:
print(dataset.shape)
print(years_shape(dataset, 'year'))

In [ ]:
dataset[dataset['temperature'].isna()]

In [ ]:
dataset

In [ ]:
dataset.groupby('year')['served_meals'].sum().reset_index()

In [ ]:
dataset['region'].unique()

In [ ]:
dataset.isna().sum()

In [ ]:
dataset.columns

In [ ]:
dataset[dataset['meal_type'] == np.nan]

In [ ]:
dataset.groupby(['week_number'])['served_meals'].sum()